In [7]:
from pykrx import stock

# 1) 지수 코드 (예: KRX 기타금융 지수)
ticker = "138040"
start, end = "20250101", "20250617"

# 2) 지수 시계열 가져오기 (시가총액, 거래량, 지수)
df = stock.get_market_cap(start, end, ticker)
# df.columns == ['시가총액', '거래량', '지수']

# 3) divisor 역산
df['divisor'] = df['시가총액'] / df['지수']

print(df[['지수','시가총액','divisor']].head())

KeyError: '지수'

In [49]:
import pandas as pd

# ① KRX에서 내려받은 WICS 분류 CSV (euc-kr 인코딩) 경로
csv_path = "data_3627_20250618.csv"
df_cls = pd.read_csv(csv_path, encoding="euc-kr")

# ② 기타금융으로 필터
df_of = df_cls[df_cls["업종명"] == "기타금융"]
other_financials = ["삼성카드", "메리츠금융지주", "iM금융지주", "에이플러스에셋", "스틱인베스트먼트", "한국토지신탁", "한국자산신탁", "SK스퀘어"]
df_of = df_of[df_of["종목명"].isin(other_financials)]

# ③ 티커 포맷 맞추기 (6자리 문자열)
tickers = df_of["종목코드"].astype(str).str.zfill(6).tolist()

In [50]:
df_of

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
167,402340,SK스퀘어,KOSPI,기타금융,153400,3500,2.33,20331767617200
196,139130,iM금융지주,KOSPI,기타금융,11710,-110,-0.93,1948460074430
393,138040,메리츠금융지주,KOSPI,기타금융,108300,-1400,-1.28,19495567425900
463,029780,삼성카드,KOSPI,기타금융,50100,-400,-0.79,5804530439100
536,026890,스틱인베스트먼트,KOSPI,기타금융,10670,-240,-2.20,444706127250
600,244920,에이플러스에셋,KOSPI,기타금융,5920,-210,-3.43,133837542560
834,123890,한국자산신탁,KOSPI,기타금융,2465,-15,-0.60,301651727590
845,034830,한국토지신탁,KOSPI,기타금융,1228,-25,-2.00,310056774440


In [51]:
tickers

['402340',
 '139130',
 '138040',
 '029780',
 '026890',
 '244920',
 '123890',
 '034830']

In [91]:
start, end = "19970101", "20250617"
df_of_mktcap = stock.get_market_cap(start, end, '034830', 'm')

In [92]:
df_of_mktcap['시가총액']

날짜
2001-05-31    345600000000
2001-06-30    311400000000
2001-07-31    302400000000
2001-08-31    234000000000
2001-09-30    174600000000
                  ...     
2025-02-28    251984251540
2025-03-31    250469316160
2025-04-30    266628626880
2025-05-31    295412399100
2025-06-30    310056774440
Freq: M, Name: 시가총액, Length: 290, dtype: int64

In [112]:
def calculate_other_financials_index(start_date: str,
                                     end_date: str,
                                     wic_csv_path: str,
                                     base_value: float = 1000.0):
    
    df_cls = pd.read_csv(wic_csv_path, encoding="euc-kr")
    df_of = df_cls[df_cls["업종명"] == "기타금융"]
    other_financials = ["삼성카드", "메리츠금융지주", "에이플러스에셋", "스틱인베스트먼트", "한국토지신탁", "한국자산신탁", "SK스퀘어"]
    #other_financials = ["삼성카드", "스틱인베스트먼트", "한국토지신탁"]
    df_of = df_of[df_of["종목명"].isin(other_financials)]
    tickers = df_of["종목코드"].astype(str).str.zfill(6).tolist()

    df_mktcap = pd.DataFrame()
    for ticker in tickers:
        df_temp = stock.get_market_cap(start_date, end_date, ticker, 'd')
        df_mktcap[ticker] = df_temp['시가총액']
        
    df_mktcap.index = pd.to_datetime(df_mktcap.index)
    
    base_sum = df_mktcap.iloc[0].sum()
    idx = df_mktcap.sum(axis=1).div(base_sum).mul(base_value)
    idx.name = "KRX_OF"
    
    return idx, df_mktcap


In [113]:
start, end = "19970701", "20250617"
wics_csv = "data_3627_20250618.csv"
of_index, df_mktcap = calculate_other_financials_index(start, end, wics_csv)
print(of_index)

날짜
2021-11-29    1000.000000
2021-11-30     937.173668
2021-12-01     901.150432
2021-12-02     947.746073
2021-12-03     939.620455
                 ...     
2025-06-11    2145.623079
2025-06-12    2146.577944
2025-06-13    2114.877271
2025-06-16    2188.284023
2025-06-17    2194.970409
Name: KRX_OF, Length: 868, dtype: float64


In [116]:
of_index.to_csv("of_index.csv", encoding="utf-8-sig")

In [115]:
df_mktcap.tail(30)

,402340,138040,029780,026890,244920,123890,034830
날짜,,,,,,,
2025-04-30,11782882276200,22339796099300,4860280477450,410530023750,106595272495,285131247580,266628626880
2025-05-02,12100980335400,22069774389800,4848694588350,395942662500,106030080170,286354986840,270415965330
2025-05-07,12233521193400,21997768600600,4842901643800,400110480000,105125772450,290026204620,269658497640
2025-05-08,12326299794000,21853757022200,4808143976500,392608408500,106821349425,290026204620,269658497640
2025-05-09,12419078394600,21277710708600,4866073422000,390524499750,107386541750,291861813510,266881116110
2025-05-12,12617889681600,21061693341000,4883452255650,405111861000,113716695790,296144900920,268648540720
2025-05-13,13240831714200,21043691893700,4866073422000,417198531750,117107849740,294921161660,269658497640
2025-05-14,13717978803000,20989687551800,4871866366550,415948186500,117560003600,294921161660,271930900710
2025-05-15,13360118486400,20503648474700,4866073422000,411363587250,117786080530,292473683140,271425922250
